## Optimizing the model

In [33]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [34]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

app1_df = application_df.drop(columns=['EIN',"NAME"])

In [35]:
# Determine the number of unique values in each column.
app1_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [36]:
# Look at APPLICATION_TYPE value counts for binning
app1_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

## Optimizing Try One

this attempt will focus on features; we will bin things differently, and try to have less values for Application type and Classification and remove the Organization type feature


In [37]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ["T12","T2","T25","T14","T29","T15","T17","T13","T9","T10"]

# Replace in dataframe
for app in application_types_to_replace:
    app1_df['APPLICATION_TYPE'] = app1_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app1_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [38]:
# Look at CLASSIFICATION value counts for binning
app1_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [39]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = pd.DataFrame(app1_df['CLASSIFICATION'].value_counts().loc[lambda x : x<2000]).index

# Replace in dataframe
for cls in classifications_to_replace:
    app1_df['CLASSIFICATION'] = app1_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app1_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [99]:
# Convert categorical data to numeric with `pd.get_dummies`
app2_df = app1_df.drop(columns = ["SPECIAL_CONSIDERATIONS","STATUS"])
 
app2_df.columns


Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [ ]:
app2_dummy = pd.get_dummies(app2_df)
app2_dummy

In [67]:
# Split our preprocessed data into our features and target arrays
fts = app2_dummy.drop('IS_SUCCESSFUL', axis = True)
tgt = app2_dummy['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(fts, tgt, random_state=12)

In [68]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [69]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu", input_dim=len(X_train_scaled[0])))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 50)                1900      
                                                                 
 dense_44 (Dense)            (None, 1)                 51        
                                                                 
Total params: 1,951
Trainable params: 1,951
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [71]:
# Train the model
m_fit = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 3s 3ms/step - loss: 0.5859 - accuracy: 0.7106
Epoch 2/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5624 - accuracy: 0.7265
Epoch 3/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5597 - accuracy: 0.7259
Epoch 4/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5578 - accuracy: 0.7269
Epoch 5/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5570 - accuracy: 0.7280
Epoch 6/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5559 - accuracy: 0.7288
Epoch 7/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5557 - accuracy: 0.7298
Epoch 8/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5549 - accuracy: 0.7296
Epoch 9/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5545 - accuracy: 0.7287
Epoch 10/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5539 - accuracy: 0.7287

In [72]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5575 - accuracy: 0.7293 - 658ms/epoch - 2ms/step
Loss: 0.5575433373451233, Accuracy: 0.7293294668197632


## TRY TWO; We will kep the data as it is (reprocessed) and play with the layers

In [73]:
nn2 = tf.keras.models.Sequential()
nn2.add(tf.keras.layers.Dense(units=25, activation="relu", input_dim=len(X_train_scaled[0])))
nn2.add(tf.keras.layers.Dense(units=50, activation="softsign"))
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn2.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 25)                950       
                                                                 
 dense_46 (Dense)            (None, 50)                1300      
                                                                 
 dense_47 (Dense)            (None, 1)                 51        
                                                                 
Total params: 2,301
Trainable params: 2,301
Non-trainable params: 0
_________________________________________________________________


In [74]:
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
m2_fit = nn2.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
804/804 [==============================] - 3s 3ms/step - loss: 0.5786 - accuracy: 0.7171
Epoch 2/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5614 - accuracy: 0.7269
Epoch 3/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5594 - accuracy: 0.7279
Epoch 4/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5573 - accuracy: 0.7275
Epoch 5/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5559 - accuracy: 0.7277
Epoch 6/25
804/804 [==============================] - 3s 4ms/step - loss: 0.5556 - accuracy: 0.7281
Epoch 7/25
804/804 [==============================] - 3s 3ms/step - loss: 0.5541 - accuracy: 0.7283
Epoch 8/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7297
Epoch 9/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7280
Epoch 10/25
804/804 [==============================] - 2s 3ms/step - loss: 0.5530 - accuracy: 0.7286

In [75]:
model_loss2, model_accuracy2 = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 1s - loss: 0.5556 - accuracy: 0.7300 - 1s/epoch - 4ms/step
Loss: 0.5556140542030334, Accuracy: 0.7300291657447815


## Try different activations and nodes

In [93]:
nn3 = tf.keras.models.Sequential()
nn3.add(tf.keras.layers.Dense(units=25, activation="selu", input_dim=len(X_train_scaled[0])))
nn3.add(tf.keras.layers.Dense(units=50, activation="selu"))
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn3.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 25)                950       
                                                                 
 dense_70 (Dense)            (None, 50)                1300      
                                                                 
 dense_71 (Dense)            (None, 1)                 51        
                                                                 
Total params: 2,301
Trainable params: 2,301
Non-trainable params: 0
_________________________________________________________________


In [94]:
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
m3_fit = nn3.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
804/804 [==============================] - 3s 3ms/step - loss: 0.5857 - accuracy: 0.7167
Epoch 2/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5682 - accuracy: 0.7241
Epoch 3/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5637 - accuracy: 0.7240
Epoch 4/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5612 - accuracy: 0.7252
Epoch 5/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5600 - accuracy: 0.7259
Epoch 6/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7257
Epoch 7/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5583 - accuracy: 0.7286
Epoch 8/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5569 - accuracy: 0.7266
Epoch 9/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5573 - accuracy: 0.7277
Epoch 10/10
804/804 [==============================] - 3s 3ms/step - loss: 0.5564 - accuracy: 0.7281

In [95]:
model_loss3, model_accuracy3 = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 1s - loss: 0.5560 - accuracy: 0.7282 - 740ms/epoch - 3ms/step
Loss: 0.5559619069099426, Accuracy: 0.7281632423400879


In [96]:
# Export our model to HDF5 file
nn2.save("AlphabetSoupCharity73_accuracy.h5")

In [98]:
models  = ["1", "2", "3"]
losses = [model_loss , model_loss2, model_loss3]
accuracy = [model_accuracy, model_accuracy2, model_accuracy3]

results = pd.DataFrame({"Model" : models, "Model Loss" : losses, "Model Accuracy" : accuracy })

results

,Model,Model Loss,Model Accuracy
0,1,0.557543,0.729329
1,2,0.555614,0.730029
2,3,0.555962,0.728163
